In [ ]:
import numpy as np 
import pandas as pd 
import random
import tensorflow as tf
import os
import seaborn as sns
import matplotlib.pyplot as plt
import optuna
from functools import partial
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score,accuracy_score,make_scorer, confusion_matrix, classification_report,accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_val_score,cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, LeakyReLU
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.layers import ELU
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.metrics import precision_recall_curve, fbeta_score

In [ ]:
# Convert data type for ANN

cov_columns1 = imputed_result.select_dtypes(include=['bool']).columns # find bool
cov_columns2 = imputed_result.select_dtypes(include=['int', 'int32', 'int64','float64']).columns # find int
cov_columns = cov_columns1.union(cov_columns2)
imputed_result[cov_columns] = imputed_result[cov_columns].astype('float32') # convert those types to float
imputed_result['hypertension'] = imputed_result['hypertension'].astype('int32') # convert those types to float

### **Split the data into train and test sets**

In [ ]:
# Split the data into training and test sets

train_data, test_data = train_test_split(
    imputed_result,
    test_size=0.2,
    random_state=42,
    stratify=imputed_result[['hypertension', 'age_group', 'male']]
)

test_male = test_data[test_data['male'] == 1]
test_female = test_data[test_data['male'] == 0]
test_age_1 = test_data[test_data['age_group'] == 1]
test_age_2 = test_data[test_data['age_group'] == 2]
test_age_3 = test_data[test_data['age_group'] == 3]
test_age_4 = test_data[test_data['age_group'] == 4]
test_age_5 = test_data[test_data['age_group'] == 5]

### **Features sets for training various models** 

In [ ]:
# Model 1 common risk factors

base_variables = [
    'male', 
    'age_group', 
    'BMI', 
    'chest_pain', 
    'angina', 
    'heart_dis', 
    'stroke', 
    'h_cholesterol', 
    'diabetes', 
    'smk'
]

# Model 2 common risk factors + psychological features

variables_2 = base_variables + [
    'anxiety',
    'sleeping_prb', 
    'happiness',
    'life_satisfaction_x'
]   

# Model 3 common risk factors + psychological features + common SES features

variables_3 = base_variables + [
    'anxiety',
    'sleeping_prb', 
    'happiness',
    'life_satisfaction_x',
    'marr_Married',
    'marr_Widow_widower',
    'marr_Never_been_married',
    'marr_Divorced_separated',
    'occu_Is pensioner',
    'occu_No employment',
    'occu_Paid employment',
    'edu_cls'
]   

# Model 4 common risk factors + psychological features + common SES features + incomes + debts

variables_4 = base_variables + [
    'anxiety',
    'sleeping_prb', 
    'happiness',
    'life_satisfaction_x',
    'marr_Married',
    'marr_Widow_widower',
    'marr_Never_been_married',
    'marr_Divorced_separated',
    'occu_Is pensioner',
    'occu_No employment',
    'occu_Paid employment',
    'edu_cls',
    'inc_cls3_Low',
    'inc_cls3_Medium',
    'inc_cls3_High',
    'inc_cls3_Very_High',
    'db_ttl2_cls_None',
    'db_ttl2_cls_Low',
    'db_ttl2_cls_Medium',
    'db_ttl2_cls_High',
    'db_ttl2_cls_Very_High'
]   

In [ ]:
# Set environment variables and random seed
def seed_everything(seed=42, use_cpu=True):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_NUM_INTEROP_THREADS'] = '1'
    os.environ['TF_NUM_INTRAOP_THREADS'] = '1'
    if use_cpu:
        os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Set environment variables before importing TensorFlow
seed_everything(42, use_cpu=True)

# Set TensorFlow random seed and configure threading
tf.random.set_seed(42)
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(1)


### **Build Artificial Neural Network model**

In [ ]:
# Model setting
def create_model(input_shape, 
                 learning_rate=0.01, 
                 l2_rate=0.01, 
                 neurons=32  
                ):
    model = Sequential([
        Input(shape=input_shape),
        # Single hidden layer with L2 regularization
        Dense(neurons, kernel_regularizer=l2(l2_rate)),
        LeakyReLU(negative_slope=0.01),   # Leaky ReLU activation function
        # Output layer: single neuron with Sigmoid activation for binary classification
        Dense(1, activation='sigmoid')
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 'auc'])
    return model

In [ ]:
# ANN with cross-validation and SMOTE
def evaluate_ann(data, 
                 predictors, 
                 target, 
                 learning_rate=0.001, 
                 l2_rate=0.01, 
                 epochs=100, 
                 batch_size=10,
                 cv_splits=5,
                 threshold=0.5,
                 k_neighbors=5,
                 neurons=10):
    X = data[predictors].values
    y = data[target].values

    # Initialize cross-validation
    kf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=42)

    # Store metrics for each fold
    auc_scores = []
    accuracy_scores = []
    f1_scores = []
    f2_scores = []
    recall_scores = []
    precision_scores = []
    y_true_all = []
    y_pred_all = []
    histories = []  # To store history of each fold

    # Start cross-validation
    for train_index, val_index in kf.split(X, y):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
    # Apply SMOTE to the training data
        smote = SMOTE(random_state=42, k_neighbors=k_neighbors)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

        # Create model
        model = create_model(input_shape=(X_train.shape[1],), 
                             learning_rate=learning_rate, 
                             l2_rate=l2_rate,
                             neurons=neurons)

        # Early stopping callback
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        # Train model with early stopping
        history = model.fit(X_train_resampled, 
                            y_train_resampled, 
                            epochs=epochs, 
                            batch_size=batch_size, 
                            verbose=0, 
                            validation_data=(X_val, y_val),
                            callbacks=[early_stopping])
        
        # Store the history
        histories.append(history.history)
        
        # Perform predictions
        pred_probs = model.predict(X_val).flatten()
        predictions = (pred_probs > threshold).astype(int)

        # Store true and predicted values for the entire dataset
        y_true_all.extend(y_val)
        y_pred_all.extend(predictions)

        # Calculate metrics for this fold
        auc_score = roc_auc_score(y_val, pred_probs)
        accuracy = accuracy_score(y_val, predictions)
        f1 = f1_score(y_val, predictions)
        f2 = fbeta_score(y_val, predictions, beta=2)
        recall = recall_score(y_val, predictions)
        precision = precision_score(y_val, predictions)

        # Store metrics
        auc_scores.append(auc_score)
        accuracy_scores.append(accuracy)
        f1_scores.append(f1)
        f2_scores.append(f2)
        recall_scores.append(recall)
        precision_scores.append(precision)

    # Calculate confusion matrix and classification report for the entire dataset
    conf_matrix = confusion_matrix(y_true_all, y_pred_all)
    report = classification_report(y_true_all, y_pred_all)

    # Calculate mean and standard deviation for other metrics
    results = {
        'Average AUC': np.mean(auc_scores),
        'AUC SD': np.std(auc_scores),
        'Average Accuracy': np.mean(accuracy_scores),
        'Accuracy SD': np.std(accuracy_scores),
        'Average F1 Score': np.mean(f1_scores),
        'F1 Score SD': np.std(f1_scores),
        'Average F2 Score': np.mean(f2_scores),
        'F2 Score SD': np.std(f2_scores),
        'Average Recall': np.mean(recall_scores),
        'Recall SD': np.std(recall_scores),
        'Average Precision': np.mean(precision_scores),
        'Precision SD': np.std(precision_scores),
        'Confusion Matrix': conf_matrix,
        'Classification Report': report,
        'Histories': histories,
        'model': model
    }

    return results
    
def print_evaluation_results(results):
    print("Evaluation Results:")
    print(f"Average AUC: {results['Average AUC']:.4f} ± {results['AUC SD']:.4f}")
    print(f"Average Accuracy: {results['Average Accuracy']:.4f} ± {results['Accuracy SD']:.4f}")
    print(f"Average F1 Score: {results['Average F1 Score']:.4f} ± {results['F1 Score SD']:.4f}")
    print(f"Average F2 Score: {results['Average F2 Score']:.4f} ± {results['F2 Score SD']:.4f}")
    print(f"Average Recall: {results['Average Recall']:.4f} ± {results['Recall SD']:.4f}")
    print(f"Average Precision: {results['Average Precision']:.4f} ± {results['Precision SD']:.4f}")
    print("\nConfusion Matrix:")
    print(results['Confusion Matrix'])
    print("\nClassification Report:")
    print(results['Classification Report'])

In [ ]:
# Evaluate random forest model on test set

def evaluate_on_test(final_model, test_data, predictors, target, threshold=0.5):
    X_test = test_data[predictors].values
    y_test = test_data[target].values

    test_pred_probs = final_model.predict(X_test).flatten()
    test_predictions = (test_pred_probs > threshold).astype(int)

    test_auc_score = roc_auc_score(y_test, test_pred_probs)
    test_accuracy = accuracy_score(y_test, test_predictions)
    test_f1 = f1_score(y_test, test_predictions)
    test_f2 = fbeta_score(y_test, test_predictions, beta=2)
    test_recall = recall_score(y_test, test_predictions)
    test_precision = precision_score(y_test, test_predictions)
    test_conf_matrix = confusion_matrix(y_test, test_predictions)
    test_report = classification_report(y_test, test_predictions)

    print("Test Results:")
    print(f"AUC: {test_auc_score:.4f}")
    print(f"Accuracy: {test_accuracy:.4f}")
    print(f"F1 Score: {test_f1:.4f}")
    print(f"F2 Score: {test_f2:.4f}")
    print(f"Recall: {test_recall:.4f}")
    print(f"Precision: {test_precision:.4f}")
    print("Confusion Matrix:")
    print(test_conf_matrix)
    print("Classification Report:")
    print(test_report)

In [ ]:
# Plot of loss and accuracy

def plot_loss_and_accuracy(histories):
    # Find the minimum number of epochs across all histories to align them
    min_epochs = min(len(h['loss']) for h in histories if 'loss' in h)

    # Truncate each history to have the same length
    for h in histories:
        for key in h.keys():
            h[key] = h[key][:min_epochs]

    # Compute average and standard deviation for each epoch across folds
    avg_loss = np.mean([h['loss'] for h in histories if 'loss' in h], axis=0)
    avg_val_loss = np.mean([h['val_loss'] for h in histories if 'val_loss' in h], axis=0)
    std_loss = np.std([h['loss'] for h in histories if 'loss' in h], axis=0)
    std_val_loss = np.std([h['val_loss'] for h in histories if 'val_loss' in h], axis=0)

    avg_accuracy = np.mean([h['accuracy'] for h in histories if 'accuracy' in h], axis=0)
    avg_val_accuracy = np.mean([h['val_accuracy'] for h in histories if 'val_accuracy' in h], axis=0)
    std_accuracy = np.std([h['accuracy'] for h in histories if 'accuracy' in h], axis=0)
    std_val_accuracy = np.std([h['val_accuracy'] for h in histories if 'val_accuracy' in h], axis=0)

    # Check if 'auc' and 'val_auc' are available in histories
    if 'auc' in histories[0] and 'val_auc' in histories[0]:
        avg_auc = np.mean([h['auc'] for h in histories if 'auc' in h], axis=0)
        avg_val_auc = np.mean([h['val_auc'] for h in histories if 'val_auc' in h], axis=0)
        std_auc = np.std([h['auc'] for h in histories if 'auc' in h], axis=0)
        std_val_auc = np.std([h['val_auc'] for h in histories if 'val_auc' in h], axis=0)

    epochs = range(1, min_epochs + 1)

    plt.figure(figsize=(18, 5))

    # Plot loss curve with confidence interval
    plt.subplot(1, 3, 1)
    plt.plot(epochs, avg_loss, label='Train Loss', color='blue')
    plt.fill_between(epochs, avg_loss - std_loss, avg_loss + std_loss, color='blue', alpha=0.2)
    plt.plot(epochs, avg_val_loss, label='Validation Loss', color='orange')
    plt.fill_between(epochs, avg_val_loss - std_val_loss, avg_val_loss + std_val_loss, color='orange', alpha=0.2)
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right')

    # Plot accuracy curve with confidence interval
    plt.subplot(1, 3, 2)
    plt.plot(epochs, avg_accuracy, label='Train Accuracy', color='blue')
    plt.fill_between(epochs, avg_accuracy - std_accuracy, avg_accuracy + std_accuracy, color='blue', alpha=0.2)
    plt.plot(epochs, avg_val_accuracy, label='Validation Accuracy', color='orange')
    plt.fill_between(epochs, avg_val_accuracy - std_val_accuracy, avg_val_accuracy + std_val_accuracy, color='orange', alpha=0.2)
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='lower right')

    # Plot AUC curve with confidence interval if available
    if 'auc' in histories[0] and 'val_auc' in histories[0]:
        plt.subplot(1, 3, 3)
        plt.plot(epochs, avg_auc, label='Train AUC', color='blue')
        plt.fill_between(epochs, avg_auc - std_auc, avg_auc + std_auc, color='blue', alpha=0.2)
        plt.plot(epochs, avg_val_auc, label='Validation AUC', color='orange')
        plt.fill_between(epochs, avg_val_auc - std_val_auc, avg_val_auc + std_val_auc, color='orange', alpha=0.2)
        plt.title('Model AUC')
        plt.ylabel('AUC')
        plt.xlabel('Epoch')
        plt.legend(loc='lower right')

    plt.tight_layout()
    plt.show()

In [ ]:
# Identify the optimal threshold balancing precision and recall

def plot_precision_recall_vs_threshold(model, data, predictors, target):
    # Ensure your model outputs probabilities (not just class labels)
    X_test = data[predictors]
    y_test = data[target]
    y_scores = model.predict(X_test).ravel()  # Flatten array if necessary

    # Calculate precision, recall, and thresholds
    precision, recall, thresholds = precision_recall_curve(y_test, y_scores)

    # Calculate F1 scores for each threshold
    # f1_scores = np.where((precision + recall) == 0, 0, 2 * (precision * recall) / (precision + recall))
    f1_scores = np.where((precision + recall) == 0, 0, 2*(precision*recall)/(precision+recall))
 
    # Find the best threshold
    best_threshold = thresholds[np.argmax(f1_scores)]
    best_f1_score = np.max(f1_scores)

    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(thresholds, precision[:-1], "b--", label="Precision")
    plt.plot(thresholds, recall[:-1], "g-", label="Recall")
    plt.plot(thresholds, f1_scores[:-1], "r-", label="F1 Score")
    plt.title("Precision, Recall, and F1 Score for different thresholds")
    plt.xlabel("Threshold")
    plt.ylabel("Score")
    plt.legend(loc="best")
    plt.grid(True)
    plt.axvline(x=best_threshold, color='gray', linestyle='--', label=f'Best Threshold: {best_threshold:.4f}')
    plt.show()

    print(f"Best F1 Score: {best_f1_score:.4f} at Threshold: {best_threshold:.4f}")

# plot_precision_recall_vs_threshold(results['model'], test_data, variables_9, 'hypertension')

In [ ]:
# Optuna for optimizing random forest parameters

def objective(trial, data, predictors, target):
    # Suggest hyperparameters to be optimized
    learning_rate = trial.suggest_categorical('learning_rate', [1e-5, 1e-4, 1e-3, 1e-2])
    l2_rate = trial.suggest_categorical('l2_rate', [1e-5, 1e-4, 1e-3, 1e-2,1e-1])
    neurons = trial.suggest_int('neurons', 10, 30, step=2)
    epochs = trial.suggest_categorical('epochs', [200])
    batch_size = trial.suggest_categorical('batch_size', [10, 15, 20])
    k_neighbors = trial.suggest_int('k_neighbors', 3,9)
    # Perform cross-validation with suggested hyperparameters
    results = evaluate_ann(data=data, 
                           predictors=predictors, 
                           target=target, 
                           learning_rate=learning_rate, 
                           l2_rate=l2_rate, 
                           epochs=epochs, 
                           batch_size=batch_size, 
                           cv_splits=5,
                           k_neighbors=k_neighbors,
                           neurons=neurons)
    return results['Average F1 Score']

def optimize_hyperparameters(data, predictors, target, n_trials=50):
    # Use partial to pass additional arguments to the objective function
    objective_func = partial(objective, data=data, predictors=predictors, target=target)

    # Create a study object and optimize the objective function
    study = optuna.create_study(direction='maximize')
    # study.optimize(objective_func, n_trials=n_trials)
    study.optimize(objective_func, n_trials=n_trials, show_progress_bar=False, catch=(Exception,), callbacks=[], n_jobs=1) #without showing the process

    # Output the best hyperparameters
    print('Best Trial:')
    trial = study.best_trial

    print(f'  Best F1 Score: {trial.value}')
    print('  Best hyperparameters:')
    for key, value in trial.params.items():
        print(f'    {key}: {value}')
    
    return trial.params

# example
# seed_everything(42, use_cpu=True)
# best_params = optimize_hyperparameters(train_data, variables_4, 'hypertension', n_trials=100)

In [ ]:
seed_everything(42, use_cpu=True)
best_params = optimize_hyperparameters(train_data, base_variables, 'hypertension', n_trials=100)

In [ ]:
best_params

In [ ]:
# Model 1 with Optuna-tuned parameters

seed_everything(42, use_cpu=True)

best_params = {'learning_rate': 0.001,
 'l2_rate': 0.001,
 'neurons': 24,
 'epochs': 200,
 'batch_size': 10,
 'k_neighbors': 5}

results = evaluate_ann(
    data=train_data, 
    predictors=base_variables, 
    target='hypertension', 
    learning_rate=best_params['learning_rate'], 
    l2_rate=best_params['l2_rate'], 
    epochs=best_params['epochs'], 
    batch_size=best_params['batch_size'],
    neurons=best_params['neurons'],
    k_neighbors=best_params['k_neighbors']
)

print_evaluation_results(results)
plot_loss_and_accuracy(results['Histories'])

In [ ]:
evaluate_on_test(results['model'], test_data,base_variables, 'hypertension',threshold = 0.5)

In [ ]:
seed_everything(42, use_cpu=True)
best_params2 = optimize_hyperparameters(train_data, variables_2, 'hypertension', n_trials=100)

In [ ]:
best_params2

In [ ]:
# Model 2 with Optuna-tuned parameters

seed_everything(42, use_cpu=True)

best_params = {'learning_rate': 0.001,
 'l2_rate': 0.01,
 'neurons': 26,
 'epochs': 200,
 'batch_size': 15,
 'k_neighbors': 5}

results = evaluate_ann(
    data=train_data, 
    predictors=variables_2, 
    target='hypertension', 
    learning_rate=best_params['learning_rate'], 
    l2_rate=best_params['l2_rate'], 
    epochs=best_params['epochs'], 
    batch_size=best_params['batch_size'],
    neurons=best_params['neurons'],
    k_neighbors=best_params['k_neighbors']
)

print_evaluation_results(results)
plot_loss_and_accuracy(results['Histories'])

In [ ]:
evaluate_on_test(results['model'], test_data,variables_2, 'hypertension',threshold = 0.5)

In [ ]:
seed_everything(42, use_cpu=True)
best_params3 = optimize_hyperparameters(train_data, variables_3, 'hypertension', n_trials=100)

In [ ]:
best_params3

In [ ]:
# Model 3 with Optuna-tuned parameters

seed_everything(42, use_cpu=True)

best_params = {'learning_rate': 0.0001,
 'l2_rate': 0.001,
 'neurons': 28,
 'epochs': 200,
 'batch_size': 20,
 'k_neighbors': 3}

results = evaluate_ann(
    data=train_data, 
    predictors=variables_3, 
    target='hypertension', 
    learning_rate=best_params['learning_rate'], 
    l2_rate=best_params['l2_rate'], 
    epochs=best_params['epochs'], 
    batch_size=best_params['batch_size'],
    neurons=best_params['neurons'],
    k_neighbors=best_params['k_neighbors']
)

print_evaluation_results(results)
plot_loss_and_accuracy(results['Histories'])

In [ ]:
evaluate_on_test(results['model'], test_data,variables_3, 'hypertension',threshold = 0.5)

In [ ]:
seed_everything(42, use_cpu=True)
best_params4 = optimize_hyperparameters(train_data, variables_4, 'hypertension', n_trials=100)
# 0.61

In [ ]:
best_params4

In [ ]:
# Model 4 with Optuna-tuned parameters

seed_everything(42, use_cpu=True)

best_params = {'learning_rate': 0.0001,
 'l2_rate': 0.0001,
 'neurons': 18,
 'epochs': 200,
 'batch_size': 10,
 'k_neighbors': 6}

results = evaluate_ann(
    data=train_data, 
    predictors=variables_4, 
    target='hypertension', 
    learning_rate=best_params['learning_rate'], 
    l2_rate=best_params['l2_rate'], 
    epochs=best_params['epochs'], 
    batch_size=best_params['batch_size'],
    neurons=best_params['neurons'],
    k_neighbors=best_params['k_neighbors']
)

print_evaluation_results(results)
plot_loss_and_accuracy(results['Histories'])

In [ ]:
evaluate_on_test(results['model'], test_data,variables_4, 'hypertension',threshold = 0.5)

In [ ]:
# Male
evaluate_on_test(results['model'], test_male,variables_4, 'hypertension',threshold = 0.5)

In [ ]:
# Female
evaluate_on_test(results['model'], test_female,variables_4, 'hypertension',threshold = 0.5)

In [ ]:
evaluate_on_test(results['model'], test_age_1,variables_4, 'hypertension',threshold = 0.5)

In [ ]:
evaluate_on_test(results['model'], test_age_2,variables_4, 'hypertension',threshold = 0.5)

In [ ]:
evaluate_on_test(results['model'], test_age_3,variables_4, 'hypertension',threshold = 0.5)

In [ ]:
evaluate_on_test(results['model'], test_age_4,variables_4, 'hypertension',threshold = 0.5)

In [ ]:
evaluate_on_test(results['model'], test_age_5,variables_4, 'hypertension',threshold = 0.5)

In [ ]:
plot_precision_recall_vs_threshold(results['model'], test_data, variables_4, 'hypertension')

In [ ]:
evaluate_on_test(results['model'], test_data,variables_4, 'hypertension',threshold = 0.4291)